In [10]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn import metrics

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import GridSearchCV,train_test_split,cross_val_score
from sklearn.model_selection import StratifiedKFold , KFold, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix, accuracy_score ,f1_score 
from sklearn.metrics import classification_report , confusion_matrix , roc_auc_score
from sklearn.preprocessing import normalize, StandardScaler ,binarize

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings("ignore")

import funciones as f
import time

!pip install graphviz
!pip install pydotplus

from IPython.display import Image
import pydotplus


import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [11]:
_random_state=40
tic_general = time.time()

In [12]:
data_covid = pd.read_csv("../data/patient_covid.csv", sep = ",", low_memory=False) 


In [13]:
X = data_covid.drop(["inpatient"],axis=1)
y = data_covid['inpatient']
#f._get_info(X)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, stratify=data_covid['inpatient'],random_state=_random_state)
display(y_train.value_counts(normalize=True).round(3))
display(y_test.value_counts(normalize=True).round(3))

print ("X_train:" , X_train.shape[0])
print ("y_train:" , y_train.shape[0])
print ("X_test:" , X_test.shape[0])
print ("y_test:" , y_test.shape[0])

0    0.735
1    0.265
Name: inpatient, dtype: float64

0    0.735
1    0.265
Name: inpatient, dtype: float64

X_train: 57146
y_train: 57146
X_test: 38098
y_test: 38098


In [15]:
#Naive Bayes      
nb=MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
score_nb = accuracy_score(y_test,y_pred_nb).round(4)

#Tree Classification
clftree = tree.DecisionTreeClassifier(random_state=_random_state)
clftree.fit(X_train, y_train)
y_pred_t = clftree.predict(X_test)
score_t = accuracy_score(y_test, y_pred_t).round(4)

#--------scaler -----------
#scaler = StandardScaler()
#X_train_std = scaler.fit_transform(X_train)
#X_test_std = scaler.transform(X_test)

#KNN (acá no especificamos n_neighbors y queda el que viene por defecto)
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
score_knn=accuracy_score(y_test, y_pred_knn).round(4)

#Logistic regression (con StandarScaler)
lr = LogisticRegression(class_weight="balance")
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
score_lr= accuracy_score(y_test, y_pred_lr).round(4)

#Scores
print ("Naive Bayes Acc Score:    \t",score_nb)
print ("KNN Acc Score:\t\t\t", score_knn)
print ("Logistic Regression Acc Score:  ", score_lr)
print ("Tree classification Acc Score:  ", score_t)

MultinomialNB()

DecisionTreeClassifier(random_state=40)

KNeighborsClassifier()

LogisticRegression(class_weight='balance')

Naive Bayes Acc Score:    	 0.8503
KNN Acc Score:			 0.8621
Logistic Regression Acc Score:   0.8738
Tree classification Acc Score:   0.8664


In [16]:
def _metric_AUC(X_t,y_t,modelo):  # return roc_auc,fpr,tpr,threshold
    probs=modelo.predict_proba(X_t)
    preds=probs[:,1]
    fpr,tpr,threshold=metrics.roc_curve(y_t, preds)
    roc_auc=metrics.auc(fpr,tpr)
    return roc_auc,fpr,tpr,threshold

# return roc_auc,fpr,tpr,threshold
#NB
roc_auc_nb,fpr_nb,tpr_nb,threshold_nb=_metric_AUC(X_test,y_test,nb)
#KNN
roc_auc_knn,fpr_knn,tpr_knn,threshold_knn=_metric_AUC(X_test,y_test,knn)
#LR
roc_auc_lr,fpr_lr,tpr_lr,threshold_lr=_metric_AUC(X_test,y_test,lr)
#T
roc_auc_t,fpr_t,tpr_t,threshold_t=_metric_AUC(X_test,y_test,clftree)
print ("ROC AUC NB:  ",  roc_auc_nb.round(4) )  #,  threshold_nb.round(4)
print ("ROC AUC KNN: ", roc_auc_knn.round(4) )
print ("ROC AUC LR:  ",  roc_auc_lr.round(4) )
print ("ROC AUC T:   ",   roc_auc_t.round(4) )

ROC AUC NB:   0.866
ROC AUC KNN:  0.8432
ROC AUC LR:   0.8912
ROC AUC T:    0.849


In [24]:
def predict_proba_ensamble(X, model_1, model_2, model_3,model_4):
    y_pred_1 = model_1.predict_proba(X)[:, 1]
    y_pred_2 = model_2.predict_proba(X)[:, 1]
    y_pred_3 = model_3.predict_proba(X)[:, 1]
    y_pred_4 = model_4.predict_proba(X)[:, 1]
    result = (y_pred_1 + y_pred_2 + y_pred_3 + y_pred_4) / 4
    return result

y_pred_ensemble = predict_proba_ensamble(X_test,clftree, nb, lr,knn)
performance_ensemble =  roc_auc_score(y_test, y_pred_ensemble)
performance_ensemble.round(4)

0.8843

In [25]:
def predict_proba_ensamble3(X, model_1, model_2, model_3):
    y_pred_1 = model_1.predict_proba(X)[:, 1]
    y_pred_2 = model_2.predict_proba(X)[:, 1]
    y_pred_3 = model_3.predict_proba(X)[:, 1]
    result = (y_pred_1 + y_pred_2 + y_pred_3 ) / 3
    return result


In [26]:
print("sin knn")
y_pred_ensemble = predict_proba_ensamble3(X_test,clftree, nb, lr)
performance_ensemble =  roc_auc_score(y_test, y_pred_ensemble)
performance_ensemble.round(4)

sin knn


0.8886

In [27]:
print("sin nb")
y_pred_ensemble = predict_proba_ensamble3(X_test,clftree, knn, lr)
performance_ensemble =  roc_auc_score(y_test, y_pred_ensemble)
performance_ensemble.round(4)

sin nb


0.8796

In [28]:
print("sin tree")
y_pred_ensemble = predict_proba_ensamble3(X_test,knn, nb, lr)
performance_ensemble =  roc_auc_score(y_test, y_pred_ensemble)
performance_ensemble.round(4)

sin tree


0.8815

In [29]:
print("sin lR")
y_pred_ensemble = predict_proba_ensamble3(X_test,knn, nb, clftree)
performance_ensemble =  roc_auc_score(y_test, y_pred_ensemble)
performance_ensemble.round(4)

sin lR


0.8789

|x|x|
|---|---|
|ROC AUC NB:|   0.866|
|ROC AUC KNN: | 0.8432|
|ROC AUC LR:  | 0.8912|
|ROC AUC T:|    0.849|

----------------------------
|x|x|
|---|---|
|los 4| 0.8843|
|sin knn | 0.8886|
|sin nb |0.8796|
|sin tree| 0.8815|
|sin lR |0.8789|
